In [2]:
from bs4 import BeautifulSoup
import requests
from nltk.tokenize import sent_tokenize
import numpy as np
import re
from operator import itemgetter
# 코사인 유사도 (1. 단어의 표현 예제 참고)
def cosine_similarity(x, y):
    
    # x와 y, 두 벡터의 코사인 유사도를 계산하는 함수
    nominator = np.dot(x, y) # 분자
    denominator = np.linalg.norm(x)*np.linalg.norm(y) # 분모
    return nominator/denominator

# 문장간 유사도 측정 (BoW를 활용 코사인 유사도 측정)
def sentence_similarity(sentence1, sentence2):
    
    # 각 문장을 소문자로 변환
    sentence1 = [word.lower() for word in sentence1.split()]
    sentence2 = [word.lower() for word in sentence2.split()]
    
    # BoW 생성을 위한 unique한 단어로 배열 생성
    words_ls = list(set(sentence1 + sentence2))
    bow1 = [0] * len(words_ls)
    bow2 = [0] * len(words_ls)
    
    # 첫번째 문장 BoW 생성
    for word in sentence1:
        bow1[words_ls.index(word)] += 1
        
    # 두번째 문장 BoW 생성
    for word in sentence2:
        bow2[words_ls.index(word)] += 1
    return cosine_similarity(bow1, bow2)

def buildMatrix(sentences):
    
    # 문장별로 그래프 edge를 Matrix 형태로 생성
    weighted_edge = np.zeros((len(sentences), len(sentences)),dtype=np.float32)
    #print(len(weighted_edge))
    #print(len(sentences))
    
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue
            weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])
            
    #print('[' + str(i) + ',' + str(j)+'] = ' + str(weighted_edge[i][j]))
    # normalize
    for i in range(len(weighted_edge)):
        
        #아래 코드 해석 = 값의 총합이 1이 된다. 이걸로 가중치를 계산, 표준화 작업.
        weighted_edge[i] /= weighted_edge[i].sum()
    
    return weighted_edge

def scoring(A, eps=0.0001, d=0.85, max_iter = 50):
    P = np.ones(len(A)) / len(A)
    print(P)
    for iter in range(0,max_iter):
        
        newP = (1 - d) + d * A.T.dot(P)
        print(newP)
        print('-------')
        
        if abs((newP - P).sum()) <= eps:
            return newP
        P = newP
    return newP


def summarizeNaverNews(url, linesinSummary = 3) :
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'html.parser')
    text = soup.find('div', class_ = '_article_body_contents').text

    text = sent_tokenize(text)
    weighted_edge = buildMatrix(text)
    score = scoring(weighted_edge)    
    #print(score)
    rankedSentenceIndexes = [item[0] for item in sorted(enumerate(score), key=lambda item: -item[1])]
    #-item은 내림차순
    #-없으면 오름차순
    #enumerate(score) = (0, socre값), (1, score값)....
    
    selectedSentences = sorted(rankedSentenceIndexes[:linesinSummary])
    summary = itemgetter(*selectedSentences)(text)
    return summary

In [20]:
pip install soynlp

Note: you may need to restart the kernel to use updated packages.


In [18]:
import re
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=008&aid=0004257362'
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'html.parser')
text = soup.find('div', class_ = '_article_body_contents').text
x = re.sub(".*\[\[.*?기자 =|▶.*",'', text)
x

'\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n\t\n 아세안지역안보포럼(ARF) 참석차 태국을 방문중인 강경화 외교부장관과 고노 다로 일본 외무상이 1일 오전(현지시간) 방콕 센타라 그랜드호텔에서 열린 양자회담에서 악수한뒤 각자의 자리로 향하고 있다.2019.8.1/뉴스1  <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>강경화 외교부 장관과 고노 다로 일본 외무상이 일본 정부의 ‘화이트리스트(백색국가)’ 제외 결정을 하루 앞둔 1일 태국 방콕에서 만났다. 두 장관이 만나는 것은 일본의 대한(對韓) 수출규제 등 경제보복 이후 처음이다. 두 장관의 태국 방문은 아세안지역안보포럼(ARF)을 비롯한 아세안 관련 외교장관회의를 위한 것이지만, 두 사람의 만남은 악화일로를 걷는 한일관계에 중대 분기점이 될 전망이다. 일본은 2일 열리는 각의(국무회의)를 통해 한국을 화이트리스트에서 제외하는 결정을 할 가능성이 크다. 배제 결정이 이뤄지면 3주 후부터 조치가 시행된다. 일본의 화이트리스트는 안보상 우호국가를 대상으로 수출관리에 우대조치를 해주는 것이다. 여기서 제외된다는 것은 한국 산업에 대한 타격뿐만 아니라 동북아 안보지형에도 적지 않은 영향을 미칠 것으로 보인다. 강 장관은 전날 방콕 태국 수완나품 공항에 도착한 직후 기자들을 만나 고노 외무상과의 회담에 대해 “화이트리스트 제외 결정 등 일본의 부당함을 지적하고 중단을 요청하겠다”고 밝혔다. 하지만 일본이 입장을 바꿀 가능성은 크지 않다는 관측이다. 한국을 화이트리스트에서 배제하는 것과 관련한 의견수렴 결과 찬성 의견이 압도적이었던 것으로 전해졌다. 고노 외무상도 이번 회담에서 일본 조치의 정당성을 주장할 것으로 보인다. 이날 회담장에 도착한 고노 외무상은 취재진이 ‘한국과 회담에 임하는 각오가 어떠냐’, ‘이번 회담이 관계개선으로 이어질 것 같으냐’고 질문했지만 아무런 답을 하지 않고 입을 굳게 다문 채 입구로 들어

In [16]:
text

"\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n\t\n\t[머니투데이 방콕(태국)=최태범  기자] [[the300]폼페이오 2일 가세, 한미일 3국 회담으로 중재나설 가능성](방콕(태국)=뉴스1) 이광호 기자 = 아세안지역안보포럼(ARF) 참석차 태국을 방문중인 강경화 외교부장관과 고노 다로 일본 외무상이 1일 오전(현지시간) 방콕 센타라 그랜드호텔에서 열린 양자회담에서 악수한뒤 각자의 자리로 향하고 있다.2019.8.1/뉴스1  <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>강경화 외교부 장관과 고노 다로 일본 외무상이 일본 정부의 ‘화이트리스트(백색국가)’ 제외 결정을 하루 앞둔 1일 태국 방콕에서 만났다. 두 장관이 만나는 것은 일본의 대한(對韓) 수출규제 등 경제보복 이후 처음이다. 두 장관의 태국 방문은 아세안지역안보포럼(ARF)을 비롯한 아세안 관련 외교장관회의를 위한 것이지만, 두 사람의 만남은 악화일로를 걷는 한일관계에 중대 분기점이 될 전망이다. 일본은 2일 열리는 각의(국무회의)를 통해 한국을 화이트리스트에서 제외하는 결정을 할 가능성이 크다. 배제 결정이 이뤄지면 3주 후부터 조치가 시행된다. 일본의 화이트리스트는 안보상 우호국가를 대상으로 수출관리에 우대조치를 해주는 것이다. 여기서 제외된다는 것은 한국 산업에 대한 타격뿐만 아니라 동북아 안보지형에도 적지 않은 영향을 미칠 것으로 보인다. 강 장관은 전날 방콕 태국 수완나품 공항에 도착한 직후 기자들을 만나 고노 외무상과의 회담에 대해 “화이트리스트 제외 결정 등 일본의 부당함을 지적하고 중단을 요청하겠다”고 밝혔다. 하지만 일본이 입장을 바꿀 가능성은 크지 않다는 관측이다. 한국을 화이트리스트에서 배제하는 것과 관련한 의견수렴 결과 찬성 의견이 압도적이었던 것으로 전해졌다. 고노 외무상도 이번 회담에서 일본 조치의 정당성을 주장할 것으로 보인다. 이날 회담장에 도착한 고노 외무상은 취재진

In [19]:
from gensim.summarization.summarizer import summarize

print(summarize(x))

아세안지역안보포럼(ARF) 참석차 태국을 방문중인 강경화 외교부장관과 고노 다로 일본 외무상이 1일 오전(현지시간) 방콕 센타라 그랜드호텔에서 열린 양자회담에서 악수한뒤 각자의 자리로 향하고 있다.2019.8.1/뉴스1  <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>강경화 외교부 장관과 고노 다로 일본 외무상이 일본 정부의 ‘화이트리스트(백색국가)’ 제외 결정을 하루 앞둔 1일 태국 방콕에서 만났다.
강 장관은 전날 방콕 태국 수완나품 공항에 도착한 직후 기자들을 만나 고노 외무상과의 회담에 대해 “화이트리스트 제외 결정 등 일본의 부당함을 지적하고 중단을 요청하겠다”고 밝혔다.
한국을 화이트리스트에서 배제하는 것과 관련한 의견수렴 결과 찬성 의견이 압도적이었던 것으로 전해졌다.
